In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text
from google.cloud.sql.connector import Connector, IPTypes
import os
from google.oauth2 import service_account



# 1. Load your local CSV data
file_path = os.path.expanduser('~/Downloads/historical-data-filtered.csv')
df = pd.read_csv(file_path)

# 2. Cleanup dates as you did before
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_datetime(df[col], dayfirst=True, format='mixed')
        except (ValueError, TypeError):
            continue

KEY_PATH = "/Users/deepshah/Downloads/tiffinstash-key.json"

credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH
)

connector = Connector(credentials=credentials)

def getconn():
    conn = connector.connect(
        "pelagic-campus-484800-b3:us-central1:tiffinstash-master",
        "pg8000",
        user="postgres",
        password="tiffinstash2026",
        db="postgres",
        ip_type=IPTypes.PUBLIC
    )
    return conn

# 4. Create the SQLAlchemy Engine
engine = sqlalchemy.create_engine("postgresql+pg8000://", creator=getconn)

# 5. Import to the new table
# We use double quotes in the name because of the hyphen
table_name = 'historical-data'

try:
    # 'if_exists=replace' will create the table if it doesn't exist
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Successfully imported {len(df)} rows into '{table_name}'")
    
    # Verification query
    with engine.connect() as conn:
        res = conn.execute(text(f'SELECT COUNT(*) FROM "{table_name}"'))
        print(f"Verified row count in DB: {res.fetchone()[0]}")

except Exception as e:
    print(f"Import failed: {e}")

# Always clean up the connector
# connector.close()

Successfully imported 247 rows into 'historical-data'
Verified row count in DB: 247


In [7]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text
from google.cloud.sql.connector import Connector, IPTypes
import os
from google.oauth2 import service_account


KEY_PATH = "/Users/deepshah/Downloads/tiffinstash-key.json"

credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH
)

connector = Connector(credentials=credentials)

def getconn():
    conn = connector.connect(
        "pelagic-campus-484800-b3:us-central1:tiffinstash-master",
        "pg8000",
        user="postgres",
        password="tiffinstash2026",
        db="postgres",
        ip_type=IPTypes.PUBLIC
    )
    return conn

# 4. Create the SQLAlchemy Engine
engine = sqlalchemy.create_engine("postgresql+pg8000://", creator=getconn)

# 5. Import to the new table
# We use double quotes in the name because of the hyphen
table_name = 'historical-data'

with engine.connect() as conn:
    df = pd.read_sql(text(f'SELECT * FROM "{table_name}"'), conn)
    print(f"Verified row count in DB: {df.shape[0]}")
pd.set_option('display.max_columns', None)
df

Verified row count in DB: 247


,ORDER ID,DATE,NAME,PHONE,EMAIL,HOUSE UNIT NO,ADDRESS LINE 1,CITY,ZIP,SKU,SELLER,DELIVERY,MEAL TYPE,MEAL PLAN,PRODUCT,PRODUCT CODE,CLABL,LABEL,DRIVER NOTE,SELLER NOTE (Changed original value),UPSTAIR DELIVERY,DELIVERY TIME,QUANTITY,DAYS,COUNT,START DATE,END DATE,STATUS,SKIP1,SKIP2,SKIP3,SKIP4,SKIP5,SKIP6,SKIP7,SKIP8,SKIP9,SKIP10,SKIP11,SKIP12,SKIP13,SKIP14,SKIP15,SKIP16,SKIP17,SKIP18,SKIP19,SKIP20,DELSAT,DELSUN,TS NOTES,DESCRIPTION
0,26646,2025-10-15,Niyati Gulati,6476136005,niyati.a.gulati@gmail.com,0,126 Huguenot Road,Oakville,L6H 0X5,LALKT-TD-MT02-W05-ONCA-LEELA,LALKT,TD,Regular,Weekly,Lal Premium Veg,MT02(LK),0,MT02(LK),Call me when arrive,0,No,LUNCH,1,5,5,p,PAUSE,PAUSE,17 Oct,21 Oct,22 Oct,23 Oct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,"start date Thu, 16 Oct, 2 tiffins remaining",• 4 Roti • 1 Veg (12 oz) • 1 Curry (12 oz)
1,26807,2025-10-21,Aakash Chokshi,4379890332,aakashchokshi8@gmail.com,303,1229 Marlborough court,Oakville,L6H 3B6,MUMMA-TD-MT09-M20-ONCA-MMJAI,MUMMA,TD,old Regular,Monthly,Mumma's Kitchen Jain,MT09(MM),Aakash Cho,Aakash Cho,"Please leave these tiffins Aakash Cho, Ugam Th...",Pure jain tiffin.,No,DINNER,1,57,57,"Sat, 21 Nov","Mon, 9 Mar",WIP,24 Nov,28 Nov,1 Dec,2 Dec,10 Dec,15 Dec,16 Dec,25 Dec,26 Dec,31 Dec,1 Jan,2 Jan,19 Dec,22 Dec,6 Jan,7 Jan,8 Jan,13 Jan,15 Jan,23 Jan,-,-,-,• 5 Roti • 1 Sabzi (08 oz) • 1 Kathod (04 oz)...
2,27020,2025-10-28,Aayush Gaur,6475808614,gaur.aayush@outlook.com,721,133 Bronte Rd,Oakville,L6L 0H2,PINDI-TD-MT10-M20-ONCA-PRPVG,PINDI,TD,Regular,Monthly,Pind Di Roti Premium Veg,MT10(PR),0,MT10(PR),Access code 0721*,0,No,LUNCH,1,20,20,p,PAUSE,PAUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,"start date Wed, 29 Oct , 10 remaining",• 4 Roti • 1 Veg (12 oz) • 1 Curry (12 oz)
3,27203,2025-11-04,Pratik Patel,7827786698,dpratik1437@gmail.com,0,2395 Skymark Avenue,Mississauga,L4W 4Y6,INFLV-TD-MTR1-W05-ONCA-OROTI,INFLV,TD,5 Roti,Weekly,India's Flavour Only Rotis,MTR1(IF),0,MTR1(IF),Call me before arrival,0,No,DINNER,1,5,5,p,PAUSE,PAUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,-,• 5 Roti Pack
4,27203,2025-11-04,Pratik Patel,7827786698,dpratik1437@gmail.com,0,2395 Skymark Avenue,Mississauga,L4W 4Y6,TSWAD-TD-MT05-W05-ONCA-FRESH,TSWAD,TD,Regular Kathod,Weekly,Swad Gujarati,MT05(SW),0,MT05(SW),Call me before arrival,0,No,DINNER,1,5,5,p,PAUSE,PAUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,-,• 5 Roti • 1 Veg (12 Oz) • 1 Kathod (12 Oz) •...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,29612-WTD,2026-01-25,Arushi Dawar,4168252967,arushidawar9@gmail.com,Unit 1901,100 Dalhousie Street,Toronto,M5B 0C7,LALKT-TD-MT35-T01-ONCA-SWAGT,LALKT,TD,Regular Rice,Trial,Lal Budget Veg,MT35(LK),0,MT35(LK),Call me when the driver reaches at the location,0,No,DINNER,1,1,1,"Mon, 26 Jan","Mon, 26 Jan",TBS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,-,• 4 Roti • 1 Veg (08 oz) • 1 Curry (08 oz) • 1...
243,29612-WTD,2026-01-25,Arushi Dawar,4168252967,arushidawar9@gmail.com,Unit 1901,100 Dalhousie Street,Toronto,M5B 0C7,FIERY-TD-MT05-T01-ONCA-FGBVG,FIERY,TD,Premium Rice,Trial,Fiery Grillss Budget Veg,MT05(FG),0,MT05(FG),Call me when the driver reaches at the location,0,No,DINNER,1,1,1,"Tue, 27 Jan","Tue, 27 Jan",TBS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,-,• 4 Roti • 1 Veg (08 oz) • 1 Curry (08 oz) • 1...
244,29612-WTD,2026-01-25,Arushi Dawar,4168252967,arushidawar9@gmail.com,Unit 1901,100 Dalhousie Street,Toronto,M5B 0C7,WAKHR-TD-MT03-T01-ONCA-WCBVG,WAKHR,TD,Regular Rice,Trial,Wakhra Chulha Budget Veg,MT03(WC),0,MT03(WC),Call me when the driver reaches at the location,0,No,DINNER,1,1,1,"Wed, 28 Jan","Wed, 28 Jan",TBS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,-,• 4 Roti • 1 Veg (08 oz) • 1 Curry (08 oz) • 1...
245,29612-WTD,2026-01-25,Arushi Dawar,4168252967,arushidawar9@gmail.com,Unit 1901,100 Dalhousie Street,Toronto,M5B 0C7,INFLV-TD-MT93-T01